
# WK03: **AWS Data Engineering Lab - Part 1**
## **Uploading Data to S3 (Separate Folders), Crawling with AWS Glue, and Querying with Athena**

### **📌 Objectives:**
1. **Manually perform each step** using the **AWS Console**.
2. **Automate the process using Boto3** in Python.
3. **Upload CSV files to separate folders in S3** to avoid merging tables.
4. **Use AWS Glue to create separate tables for each dataset**.
5. **Query the data using AWS Athena**.
6. **Set appropriate IAM policies for Glue and Athena**.

---

## **🔹 Part 1: Manual Steps Using the AWS Console**

### **1️⃣ Upload Data to S3 (AWS Console)**
1. Go to the **AWS Management Console** → Open **S3**.
2. Click **Create Bucket**, set a **globally unique name**, choose a **region**, and click **Create**.
3. Click on the newly created **S3 bucket**.
4. Click **Create Folder** → Name it `population_data/`.
5. Click **Create Folder** → Name it `economic_data/`.
6. Click into each folder and **Upload** the corresponding CSV file:
   - **Upload** `population_data.csv` into `population_data/`.
   - **Upload** `economic_data.csv` into `economic_data/`.

### **2️⃣ Set Up AWS Glue Crawler (AWS Console)**
1. Go to **AWS Glue Console** → Click **Crawlers** → Click **Create Crawler**.
2. Name the first crawler **population_crawler**.
3. Select **S3** as the source → Enter **S3 bucket path**: `s3://your-bucket/datasets/population_data/`.
4. Choose **IAM Role** (`AWSGlueServiceRole`) with necessary permissions.
5. Select **"Create a new database"** → Name it (`policy_data_catalog`).
6. Click **Create and Run**.
7. Repeat the same process for **economic_crawler** with `s3://your-bucket/datasets/economic_data/`.

### **3️⃣ Query Data Using AWS Athena (AWS Console)**
1. Go to **AWS Athena Console**.
2. Set the **Query Results Location** to an **S3 bucket**.
3. Select **Database: policy_data_catalog**.
4. Run the query:
   ```sql
   SELECT * FROM population_data LIMIT 10;
   ```
5. Run a second query:
   ```sql
   SELECT * FROM economic_data LIMIT 10;
   ```
6. Review query results.

---

## **🔹 Part 2: Automating the Process Using Boto3**

### **1️⃣ Upload Data to S3 Using Boto3 (Separate Folders)**
We will now **automate the S3 upload** by specifying a **directory** and uploading files to separate folders.



In [ ]:

import boto3
import os

# Initialize S3 client
s3_client = boto3.client('s3')

# Define S3 bucket and directory containing files
bucket_name = "your-bucket-name"
local_directory = "./data_files/"  # Change this to your directory path

# Define files and their respective folders in S3
files_and_folders = {
    "population_data.csv": "datasets/population_data/",
    "economic_data.csv": "datasets/economic_data/"
}

# Upload files to S3
for file, s3_folder in files_and_folders.items():
    file_path = os.path.join(local_directory, file)
    s3_client.upload_file(file_path, bucket_name, f"{s3_folder}{file}")
    print(f"Uploaded {file} to s3://{bucket_name}/{s3_folder}{file}")



### **2️⃣ Set Up AWS Glue Crawlers Using Boto3**
The following script will **create a Glue database** and a **Glue Crawler for each dataset** to ensure separate tables.


In [ ]:

glue_client = boto3.client('glue')

# Define Glue database name
glue_database = "policy_data_catalog"
crawler_population = "population_crawler"
crawler_economic = "economic_crawler"

# Create Glue Database
glue_client.create_database(DatabaseInput={'Name': glue_database})
print(f"Glue database '{glue_database}' created.")

# Create Glue Crawlers for separate datasets
for crawler_name, s3_path in [(crawler_population, "datasets/population_data/"), (crawler_economic, "datasets/economic_data/")]:
    glue_client.create_crawler(
        Name=crawler_name,
        Role="AWSGlueServiceRole",  # Ensure this IAM role has access to S3 and Glue
        DatabaseName=glue_database,
        Targets={"S3Targets": [{"Path": f"s3://{bucket_name}/{s3_path}"}]},
        TablePrefix=f"{crawler_name}_"
    )
    glue_client.start_crawler(Name=crawler_name)
    print(f"Started Glue Crawler '{crawler_name}'. Wait for it to complete before proceeding.")



### **3️⃣ Query Data Using AWS Athena with Boto3**
Now that the data is registered in Glue, we can **query it using Athena**.


In [ ]:

athena_client = boto3.client('athena')

# Define Athena queries for both datasets
queries = {
    "population_data": "SELECT * FROM population_crawler_population_data LIMIT 10;",
    "economic_data": "SELECT * FROM economic_crawler_economic_data LIMIT 10;"
}

output_location = f"s3://{bucket_name}/athena-results/"

# Execute queries
for table_name, query in queries.items():
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={"Database": glue_database},
        ResultConfiguration={"OutputLocation": output_location}
    )
    query_execution_id = response['QueryExecutionId']
    print(f"Athena Query for {table_name} Execution ID: {query_execution_id}. Results will be in {output_location}")
